In [7]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("diabetic_data.csv")

df.replace("?", np.nan, inplace=True)
df.dropna(subset=["diag_1", "diag_2", "diag_3"], inplace=True)

In [6]:
df["readmitted_30"] = df["readmitted"].apply(lambda x: 1 if x == "<30" else 0)

In [9]:
categorical_cols = df.select_dtypes(include="object").columns

le = LabelEncoder()
for col in categorical_cols:
    df[col] = le.fit_transform(df[col].astype(str))